<a href="https://colab.research.google.com/github/MayerT1/Forest_Structure_Function_Module/blob/main/Step_3_Forest_Structure_Function_Module_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Export CSVs with this GEE Script

https://code.earthengine.google.com/ca98e825537939f29d0a7703cdf51134

guide: https://www.kaggle.com/code/sociopath00/random-forest-using-gridsearchcv

# set-up

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/sample_partition

In [ ]:
!ls

In [ ]:
!pip3 install mxnet-mkl==1.6.0 numpy==1.23.1

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

from subprocess import check_output
# print(check_output(["ls", "../input"]).decode("utf8"))

%matplotlib inline

In [ ]:
train=pd.read_csv("training.csv")
# print("Train dataset has {} samples and {} attributes".format(*train.shape))

sns.histplot(data=train, x="rh100")

train.describe()

In [ ]:

# Example: If you have a DataFrame 'df' with 'rh100' as the target column.
def plot_scatter_with_rh100(train, target_column='rh100'):
    # Check if 'rh100' exists in the DataFrame
    if target_column not in train.columns:
        raise ValueError(f"'{target_column}' is not a column in the DataFrame.")

    # Set up the number of subplots based on the number of columns (excluding 'rh100')
    features = train.drop(columns=[target_column]).columns
    num_cols = len(features)
    num_rows = (num_cols // 3) + 1  # Customize for more/less features per row

    fig, axes = plt.subplots(num_rows, 3, figsize=(15, 5 * num_rows))  # Adjust figure size
    axes = axes.flatten()  # Flatten the axes array for easy indexing

    for i, column in enumerate(features):
        sns.scatterplot(x=train[column], y=train[target_column], ax=axes[i])
        axes[i].set_title(f"{column} vs {target_column}")
        axes[i].set_xlabel(column)
        axes[i].set_ylabel(target_column)

    # Remove any unused subplots
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()
    plt.show()

# Example: df = pd.read_csv('your_dataset.csv')
plot_scatter_with_rh100(train)

In [ ]:
test=pd.read_csv("testing.csv")
print("Test dataset has {} samples and {} attributes".format(*test.shape))

In [ ]:
validation=pd.read_csv("validation.csv")
print("validation dataset has {} samples and {} attributes".format(*validation.shape))

# Start

In [ ]:
head_testlist = list(test.columns.values)
head_testlist

In [ ]:
# test.drop([0, 39],axis=1)
test.drop(['system:index', 'class', '.geo'], axis=1, inplace=True)
test

In [ ]:
test.head()

In [ ]:
train.head()

In [ ]:
head_list = list(train.columns.values)
head_list

In [ ]:
fig , ax = plt.subplots(figsize=(6,4))
sns.countplot(x='class', data=train)
plt.title("class plot")
plt.show()

In [ ]:

del head_list[0]
del head_list[22]
del head_list[36]

head_list

In [ ]:
corr_df=train[head_list]  #New dataframe to calculate correlation between numeric features
cor= corr_df.corr(method='pearson')
print(cor)

In [ ]:
fig, ax =plt.subplots(figsize=(10, 10))
plt.title("Correlation Plot")
sns.heatmap(cor, mask=np.zeros_like(cor), cmap=sns.diverging_palette(220, 10, as_cmap=True),
            square=True, ax=ax)
plt.show()

In [ ]:
# csq=chi2_contingency(pd.crosstab(train['class'], train['replace_w_categorical_variables']))
# print("P-value: ",csq[1])

In [ ]:
fig, ax=plt.subplots(1,figsize=(8,6))
sns.boxplot(x='class',y='rh100_1', data=train)
ax.set_ylim(0,40)
plt.title("class vs rh100_1")
plt.show()

In [ ]:
train['rh100_1'].describe()

# Model

In [ ]:
y=train['rh100']
y

In [ ]:
df = train.drop(['system:index', 'class', 'rh100', 'rh100_1', '.geo'], axis=1, inplace=True)
df

In [ ]:
df= train
df

In [ ]:
head_dflist = list(df.columns.values)
head_dflist

# SFS

In [ ]:
# rfc=RandomForestClassifier(random_state=42)

In [ ]:
# RandomForestRegressor
# class sklearn.ensemble.RandomForestRegressor(n_estimators=100, *, criterion='squared_error', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=1.0, max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, ccp_alpha=0.0, max_samples=None, monotonic_cst=None)[source]


In [ ]:
from sklearn.ensemble import RandomForestRegressor

rfr=RandomForestRegressor(random_state=42)

In [ ]:
# x_train, x_test, y_train, y_test = train_test_split(df, y, test_size=0.3, random_state=42)

In [ ]:
x_train = df
y_train = y

In [ ]:
# from sklearn.feature_selection import SequentialFeatureSelector
# # from sklearn.neighbors import KNeighborsClassifier
# # from sklearn.datasets import load_iris
# # X, y = load_iris(return_X_y=True)
# # knn = KNeighborsClassifier(n_neighbors=3)
# sfs = SequentialFeatureSelector(rfr,  n_features_to_select=36, tol = float, scoring='accuracy')
# sfs.fit(x_train, y_train)
# # sfs.get_support()
# # sfs.transform(x_train).shape

# selected_features = sfs.get_support()

# count = len(list(x_train.columns[selected_features]))
# count

# print('The selected features are:', list(x_train.columns[selected_features]))

In [ ]:
!pip install mlxtend

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

#SequentialFeatureSelector(estimator, k_features=1, forward=True, floating=False, verbose=0, scoring=None, cv=5,
# n_jobs=1, pre_dispatch='2n_jobs', clone_estimator=True, fixed_features=None, feature_groups=None)*

#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

scoring = {'neg_mean_absolute_error','r2'}

# Sequential Forward Floating Selection
sfs = SFS(rfr,
          k_features= 4,
          forward=True,
          floating=True,
          verbose=3,
          scoring='neg_mean_absolute_error',
          cv=5,)


sfs.fit(x_train, y_train)
sfs.subsets_
sfs.k_feature_idx_


In [ ]:
print('Best accuracy score: %.2f' % sfs.k_score_)
print('Best subset (indices):', sfs.k_feature_idx_)
print('Best subset (corresponding names):', sfs.k_feature_names_)

In [ ]:
sfs_df = pd.DataFrame.from_dict(sfs.get_metric_dict()).T
sfs_df

In [ ]:
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs

fig1 = plot_sfs(sfs.get_metric_dict(), kind='std_dev')

plt.ylim([-10, 1])
plt.title('Sequential Forward Floating Selection (w. std_dev)')
plt.grid()
plt.show()

# RF Train

In [ ]:
#### pull the SFS best features and make that new x_train

best_features = list(sfs.k_feature_names_)
best_features


In [ ]:
best_features_df = df[best_features]
x_train =best_features_df
x_train

In [ ]:
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
param_grid = {
    'n_estimators': [10, 50, 100, 150], #10, 50, 100
    # 'max_features': ['auto', 'sqrt', 'log2'],
    # 'max_depth' : [7,8],
    # 'min_samples_split': [3,4],
    'criterion' :['absolute_error', 'squared_error'] #'squared_error', 'friedman_mse'
}

In [ ]:
CV_rfc = GridSearchCV(estimator=rfr, param_grid=param_grid, scoring = 'r2', cv= 5, verbose = 3)
CV_rfc.fit(x_train, y_train)

In [ ]:
print(CV_rfc.best_score_)
print(CV_rfc.best_params_)
print(CV_rfc.best_estimator_)

In [ ]:
sorted(CV_rfc.cv_results_.keys())

In [ ]:
cv_df = pd.DataFrame(CV_rfc.cv_results_)
cv_df

In [ ]:
from matplotlib import pyplot as plt
cv_df.plot(kind='scatter', x='param_n_estimators', y='mean_test_score', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
cv_df_out = pd.DataFrame(CV_rfc.cv_results_['params'])

cv_df_out['score'] = CV_rfc.cv_results_['mean_test_score']

sns.lineplot(data=cv_df_out, x= 'n_estimators', y='score') #hue='param_kernel'
plt.show()

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(cv_df['param_criterion'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(cv_df, x='mean_test_score', y='param_criterion', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

# Run the model

In [ ]:
# print(CV_rfc.best_params_)
# model_input = CV_rfc.best_params_

rfr1=RandomForestRegressor(random_state=42, min_samples_split= 3, n_estimators= 50, max_depth=7, criterion='absolute_error', verbose = 3)


In [ ]:
rfr1.fit(x_train, y_train)

In [ ]:
test_best_features_df = test[best_features]
x_test  = test_best_features_df
x_test

In [ ]:
pred=rfr1.predict(x_test)

In [ ]:
y_test=test['rh100']
y_test

In [ ]:
fig, ax = plt.subplots()
ax.scatter(y_test, pred)
ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4)
ax.set_xlabel('Actual')
ax.set_ylabel('Predicted')
# #regression line

plt.annotate("r-squared = {:.3f}".format(r2_score(y_test, pred)), (0, 1))
plt.show()

In [ ]:
rf_test_plot_df = pd.DataFrame({'y_test':y_test, 'pred':pred})
# rf_test_plot_df.plot('y_test', 'pred', kind='scatter')

sns.regplot(data=rf_test_plot_df,x = 'y_test', y = 'pred',  ci=99, marker="x", color=".3", line_kws=dict(color="r"))

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


print("MAE train on test",mean_absolute_error(y_test,pred))

print("MSE train on test",mean_squared_error(y_test,pred))

r2 = r2_score(y_test,pred)
print("R2 train on test", r2)

print("RMSE train on test",np.log(np.sqrt(mean_squared_error(y_test,pred))))

# Validation

In [ ]:
val_best_features_df = validation[best_features]
x_val = val_best_features_df
x_val

In [ ]:
# validation

y_val=validation['rh100']
y_val

In [ ]:
pred_val=rfr1.predict(x_val)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(y_val, pred_val)
ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4)
ax.set_xlabel('Actual')
ax.set_ylabel('Predicted')
# #regression line

plt.annotate("r-squared = {:.3f}".format(r2_score(y_val, pred_val)), (0, 1))
plt.show()

In [ ]:
rf_val_plot_df = pd.DataFrame({'y_val':y_val, 'pred_val':pred_val})
# rf_test_plot_df.plot('y_test', 'pred', kind='scatter')

sns.regplot(data=rf_val_plot_df,x = 'y_val', y = 'pred_val',  ci=99, marker="x", color=".3", line_kws=dict(color="r"))

In [ ]:
print("MAE on val",mean_absolute_error(y_val,pred_val))

print("MSE  on val",mean_squared_error(y_val,pred_val))

r2 = r2_score(y_val,pred_val)
print("R2  on val", r2)

print("RMSE  on val",np.log(np.sqrt(mean_squared_error(y_val,pred_val))))

In [ ]:
# fig, ax = plt.subplots()
# ax.scatter(y_val, pred)
# ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4)
# ax.set_xlabel('Actual')
# ax.set_ylabel('Predicted')
# # #regression line

# plt.annotate("r-squared = {:.3f}".format(r2_score(y_val, pred)), (0, 1))
# plt.show()

In [ ]:
# rf_val_plot_df = pd.DataFrame({'y_val':y_val, 'pred':pred})
# # rf_test_plot_df.plot('y_test', 'pred', kind='scatter')

# sns.regplot(data=rf_val_plot_df,x = 'y_val', y = 'pred',  ci=99, marker="x", color=".3", line_kws=dict(color="r"))

In [ ]:

# print("MAE on val",mean_absolute_error(y_val,pred))

# print("MSE  on val",mean_squared_error(y_val,pred))

# r2 = r2_score(y_val,pred)
# print("R2  on val", r2)

# print("RMSE  on val",np.log(np.sqrt(mean_squared_error(y_val,pred))))

to follow up on: https://www.kaggle.com/code/wrecked22/randomforest-r-squared-0-86

# Print csv

updated needed here

In [ ]:
# op=pd.DataFrame(test['rh100_1'])
# op['class']=op_rf

In [ ]:
# op.to_csv("op_rf.csv", index=False)

In [ ]:
# from pathlib import Path
# filepath = Path('/content/drive/MyDrive/sample_partition/out.csv')
# filepath.parent.mkdir(parents=True, exist_ok=True)
# df.to_csv(filepath)